[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aurelio-labs/langchain-course/blob/main/chapters/07-lcel.ipynb)

#### LangChain Essentials Course

# LangChains Expression Language

LangChain is one of the most popular open source libraries for AI Engineers. It's goal is to abstract away the complexity in building AI software, provide easy-to-use building blocks, and make it easier when switching between AI service providers.

In this example, we will introduce LangChain's Expression Langauge (LCEL), abstracting a full chain and understanding how it will work. We'll provide examples for both OpenAI's `gpt-4o-mini` *and* Meta's `llama3.2` via Ollama!

In [1]:
!pip install -qU \
  langchain-core \
  langchain-openai\
  langchain-community \
  langsmith \
  docarray

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.6/437.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.3/360.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.4 MB/s eta 0:00:00


---

> ⚠️ We will be using OpenAI for this example allowing us to run everything via API. If you would like to use Ollama instead, check out the [Ollama LangChain Course](https://github.com/aurelio-labs/langchain-course/tree/main/notebooks/ollama).

---

---

> ⚠️ If using LangSmith, add your API key below:

In [2]:
import os
from getpass import getpass

os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") or \
    getpass("Enter LangSmith API Key: ")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "aurelioai-langchain-course-lcel-openai"

Enter LangSmith API Key: ··········


---

## Traditional Chains vs LCEL

In this section we're going to dive into a basic example using the traditional method for building chains before jumping into LCEL. We will build a pipeline where the user must input a specific topic, and then the LLM will look and return a report on the specified topic. Generating a _research report_ for the user.

### Traditional LLMChain

The `LLMChain` is the simplest chain originally introduced in LangChain. This chain takes a prompt, feeds it into an LLM, and _optionally_ adds an output parsing step before returning the result.

Let's see how we construct this using the traditional method, for this we need:

* `prompt` — a `PromptTemplate` that will be used to generate the prompt for the LLM.
* `llm` — the LLM we will be using to generate the output.
* `output_parser` — an optional output parser that will be used to parse the structured output of the LLM.

In [3]:
from langchain import PromptTemplate

prompt_template = "Give me a small report on {topic}"

prompt = PromptTemplate(
    input_variables=["topic"],
    template=prompt_template
)

For the LLM, we'll start by initializing our connection to the OpenAI API. We do need an OpenAI API key, which you can get from the [OpenAI platform](https://platform.openai.com/api-keys).

We will use the `gpt-4o-mini` model with a `temperature` of `0.0`:

In [4]:
import os
from getpass import getpass
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") \
    or getpass("Enter your OpenAI API key: ")

llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.0,
)

Enter your OpenAI API key: ··········


In [5]:
llm_out = llm.invoke("Hello there")
llm_out

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 9, 'total_tokens': 19, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BUIvSlHFoiyUpdYpvnVlNeISJsbD7', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--79b93df6-4173-472b-b36a-431ee93b5a03-0', usage_metadata={'input_tokens': 9, 'output_tokens': 10, 'total_tokens': 19, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Then we define our output parser, this will be used to parse the output of the LLM. In this case, we will use the `StrOutputParser` which will parse the `AIMessage` output from our LLM into a single string.

In [6]:
from langchain.schema.output_parser import StrOutputParser

output_parser = StrOutputParser()

In [7]:
out = output_parser.invoke(llm_out)
out

'Hello! How can I assist you today?'

Through the `LLMChain` class we can place each of our components into a linear `chain`.

In [8]:
from langchain.chains import LLMChain

chain = LLMChain(prompt=prompt, llm=llm, output_parser=output_parser)

<ipython-input-8-32987ff2088c>:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(prompt=prompt, llm=llm, output_parser=output_parser)


Note that the `LLMChain` _was_ deprecated in LangChain `0.1.17`, the expected way of constructing these chains today is through LCEL, which we'll cover in a moment.

We can `invoke` our `chain`, providing a `topic` that we'd like to be researched.

In [9]:
result = chain.invoke("retrieval augmented generation")
result

{'topic': 'retrieval augmented generation',
 'text': "### Report on Retrieval-Augmented Generation (RAG)\n\n#### Introduction\nRetrieval-Augmented Generation (RAG) is an innovative approach that combines the strengths of information retrieval and natural language generation. This method enhances the capabilities of language models by allowing them to access external knowledge bases or documents during the generation process, thereby improving the accuracy and relevance of the generated content.\n\n#### Key Concepts\n1. **Information Retrieval**: RAG utilizes a retrieval mechanism to fetch relevant documents or snippets from a large corpus of text. This step is crucial as it provides the model with up-to-date and contextually relevant information that may not be encoded in its training data.\n\n2. **Natural Language Generation**: After retrieving relevant information, the model generates responses or text based on both the retrieved content and the input query. This dual approach allows

We can view a formatted version of this output using the `Markdown` display:

In [10]:
from IPython.display import display, Markdown

display(Markdown(result["text"]))

### Report on Retrieval-Augmented Generation (RAG)

#### Introduction
Retrieval-Augmented Generation (RAG) is an innovative approach that combines the strengths of information retrieval and natural language generation. This method enhances the capabilities of language models by allowing them to access external knowledge bases or documents during the generation process, thereby improving the accuracy and relevance of the generated content.

#### Key Concepts
1. **Information Retrieval**: RAG utilizes a retrieval mechanism to fetch relevant documents or snippets from a large corpus of text. This step is crucial as it provides the model with up-to-date and contextually relevant information that may not be encoded in its training data.

2. **Natural Language Generation**: After retrieving relevant information, the model generates responses or text based on both the retrieved content and the input query. This dual approach allows for more informed and contextually appropriate outputs.

3. **Architecture**: RAG typically consists of two main components:
   - **Retriever**: This component identifies and retrieves relevant documents from a pre-defined knowledge base or dataset.
   - **Generator**: This component, often based on transformer architectures like BERT or GPT, generates coherent and contextually relevant text using the retrieved information.

#### Advantages
- **Enhanced Accuracy**: By leveraging external knowledge, RAG can produce more accurate and factually correct responses, especially in domains where information is constantly evolving.
- **Contextual Relevance**: The integration of retrieval allows the model to maintain context and relevance, improving the quality of generated text.
- **Scalability**: RAG can be scaled to incorporate vast amounts of data, making it suitable for applications requiring extensive knowledge bases.

#### Applications
- **Question Answering**: RAG is particularly effective in question-answering systems where users seek specific information that may not be present in the model's training data.
- **Chatbots and Virtual Assistants**: By providing real-time information, RAG enhances the conversational abilities of chatbots, making them more useful and informative.
- **Content Creation**: RAG can assist in generating articles, reports, or summaries by pulling in relevant data from various sources.

#### Challenges
- **Complexity**: The integration of retrieval and generation components adds complexity to the system, requiring careful tuning and optimization.
- **Dependence on Quality of Data**: The effectiveness of RAG is heavily reliant on the quality and relevance of the retrieved documents. Poor retrieval can lead to inaccurate or irrelevant generation.
- **Latency**: The retrieval process can introduce latency, which may affect the responsiveness of applications, particularly in real-time scenarios.

#### Conclusion
Retrieval-Augmented Generation represents a significant advancement in the field of natural language processing. By combining retrieval and generation, RAG enhances the ability of language models to produce accurate, relevant, and contextually rich text. As research and development in this area continue, RAG is poised to play a crucial role in various applications, from customer support to content generation, ultimately improving user experience and information accessibility.

That is a simple `LLMChain` using the traditional LangChain method. Now let's move onto LCEL.

## LangChain Expression Language (LCEL)

**L**ang**C**hain **E**xpression **L**anguage (LCEL) is the recommended approach to building chains in LangChain. Having superceeded the traditional methods with `LLMChain`, etc. LCEL gives us a more flexible system for building chains. The pipe operator `|` is used by LCEL to _chain_ together components. Let's see how we'd construct an `LLMChain` using LCEL.

In [11]:
lcel_chain = prompt | llm | output_parser

We can `invoke` this chain in the same way as we did before:

In [12]:
result = lcel_chain.invoke("retrieval augmented generation")
result

'### Report on Retrieval-Augmented Generation (RAG)\n\n#### Introduction\nRetrieval-Augmented Generation (RAG) is an innovative approach that combines the strengths of information retrieval and natural language generation. This method enhances the capabilities of language models by allowing them to access external knowledge sources, thereby improving the accuracy and relevance of generated responses.\n\n#### Concept Overview\nRAG operates on the principle of integrating a retrieval mechanism with a generative model. The process typically involves two main components:\n\n1. **Retrieval Component**: This part of the system retrieves relevant documents or pieces of information from a large corpus based on a given query. It often employs techniques such as dense vector representations and similarity search to identify the most pertinent data.\n\n2. **Generation Component**: After retrieving relevant information, the generative model (often based on architectures like Transformers) synthesi

The output format is slightly different, but the underlying functionality and content being output is the same. As before, we can view a formatted version of this output using the `Markdown` display:

In [13]:
display(Markdown(result))

### Report on Retrieval-Augmented Generation (RAG)

#### Introduction
Retrieval-Augmented Generation (RAG) is an innovative approach that combines the strengths of information retrieval and natural language generation. This method enhances the capabilities of language models by allowing them to access external knowledge sources, thereby improving the accuracy and relevance of generated responses.

#### Concept Overview
RAG operates on the principle of integrating a retrieval mechanism with a generative model. The process typically involves two main components:

1. **Retrieval Component**: This part of the system retrieves relevant documents or pieces of information from a large corpus based on a given query. It often employs techniques such as dense vector representations and similarity search to identify the most pertinent data.

2. **Generation Component**: After retrieving relevant information, the generative model (often based on architectures like Transformers) synthesizes a coherent and contextually appropriate response. This model uses the retrieved documents as additional context to enhance its output.

#### Advantages
- **Enhanced Knowledge Access**: RAG allows models to leverage vast external knowledge bases, making them more informed and capable of providing accurate answers.
- **Improved Contextuality**: By incorporating real-time data retrieval, RAG can generate responses that are more relevant to current events or specific user queries.
- **Reduced Hallucination**: Traditional generative models sometimes produce inaccurate or fabricated information (a phenomenon known as "hallucination"). RAG mitigates this risk by grounding responses in retrieved documents.

#### Applications
RAG has a wide range of applications, including:
- **Question Answering**: Providing precise answers to user queries by retrieving relevant information from databases or the internet.
- **Chatbots and Virtual Assistants**: Enhancing conversational agents with up-to-date knowledge and contextually relevant responses.
- **Content Creation**: Assisting in generating articles, reports, or summaries by pulling in relevant data from various sources.

#### Challenges
Despite its advantages, RAG faces several challenges:
- **Retrieval Quality**: The effectiveness of the system heavily relies on the quality of the retrieval component. Poor retrieval can lead to irrelevant or misleading information being used in generation.
- **Computational Complexity**: The dual process of retrieval and generation can be resource-intensive, requiring significant computational power and optimization.
- **Integration of Diverse Sources**: Ensuring that the retrieved information is coherent and contextually integrated into the generated text can be complex.

#### Conclusion
Retrieval-Augmented Generation represents a significant advancement in the field of natural language processing. By effectively combining retrieval and generation, RAG systems can produce more accurate, relevant, and context-aware outputs. As research and development in this area continue, RAG is poised to play a crucial role in the evolution of intelligent systems and applications across various domains.

### How Does the Pipe Operator Work?

Before moving onto other LCEL features, let's take a moment to understand what the pipe operator `|` is doing and _how_ it works.

Functionality wise, the pipe tells you that whatever the _left_ side outputs will be fed as input into the _right_ side. In the example of `prompt | llm | output_parser`, we see that `prompt` feeds into `llm` feeds into `output_parser`.

The pipe operator is a way of chaining together components, and is a way of saying that whatever the _left_ side outputs will be fed as input into the _right_ side.

Let's make a basic class named `Runnable` that will transform our a provided function into a _runnable_ class that we will then use with the pipe `|` operator.

In [14]:
class Runnable:
    def __init__(self, func):
        self.func = func
    def __or__(self, other):
        def chained_func(*args, **kwargs):
            return other.invoke(self.func(*args, **kwargs))
        return Runnable(chained_func)
    def invoke(self, *args, **kwargs):
        return self.func(*args, **kwargs)

With the `Runnable` class, we will be able wrap a function into the class, allowing us to then chain together multiple of these _runnable_ functions using the `__or__` method.

First, let's create a few functions that we'll chain together:

In [15]:
def add_five(x):
    return x+5

def sub_five(x):
    return x-5

def mul_five(x):
    return x*5

Now we wrap our functions with the `Runnable`:

In [16]:
add_five_runnable = Runnable(add_five)
sub_five_runnable = Runnable(sub_five)
mul_five_runnable = Runnable(mul_five)

Finally, we can chain these together using the `__or__` method from the `Runnable` class:

In [17]:
chain = (add_five_runnable).__or__(sub_five_runnable).__or__(mul_five_runnable)

chain.invoke(3)

15

So we can see that we're able to chain together our functions using `__or__`. The pipe `|` operator is simply a shortcut for the `__or__` method, so we can create the exact same chain like so:

In [18]:
chain = add_five_runnable | sub_five_runnable | mul_five_runnable

chain.invoke(3)

15

## LCEL `RunnableLambda`

The `RunnableLambda` class is LangChain's built-in method for constructing a _runnable_ object from a function. That is, it does the same thing as the custom `Runnable` class we created earlier. Let's try it out with the same functions as before.

In [19]:
from langchain_core.runnables import RunnableLambda

add_five_runnable = RunnableLambda(add_five)
sub_five_runnable = RunnableLambda(sub_five)
mul_five_runnable = RunnableLambda(mul_five)

We chain these together again with the pipe `|` operator:

In [20]:
chain = add_five_runnable | sub_five_runnable | mul_five_runnable

And call them using the `invoke` method:

In [21]:
chain.invoke(3)

15

Now we want to try something a little more testing, so this time we will generate a report, and we will try and edit that report using this functionallity.

In [22]:
prompt_str = "give me a small report about {topic}"
prompt = PromptTemplate(
    input_variables=["topic"],
    template=prompt_str
)

In [23]:
chain = prompt | llm | output_parser

In [24]:
result = chain.invoke("AI")
display(Markdown(result))

### Report on Artificial Intelligence (AI)

#### Introduction
Artificial Intelligence (AI) refers to the simulation of human intelligence in machines programmed to think and learn like humans. It encompasses a variety of technologies and methodologies, including machine learning, natural language processing, robotics, and computer vision. AI has rapidly evolved over the past few decades, becoming an integral part of various industries and daily life.

#### Current State of AI
As of 2023, AI technologies are being utilized across multiple sectors, including healthcare, finance, transportation, and entertainment. Key advancements include:

1. **Machine Learning (ML)**: Algorithms that enable computers to learn from and make predictions based on data. Deep learning, a subset of ML, has led to breakthroughs in image and speech recognition.

2. **Natural Language Processing (NLP)**: Technologies that allow machines to understand and respond to human language. Applications include chatbots, virtual assistants, and language translation services.

3. **Computer Vision**: The ability of machines to interpret and make decisions based on visual data. This technology is widely used in facial recognition, autonomous vehicles, and medical imaging.

4. **Robotics**: AI-driven robots are increasingly used in manufacturing, logistics, and even healthcare, performing tasks ranging from assembly line work to surgery.

#### Applications of AI
- **Healthcare**: AI is used for diagnostics, personalized medicine, and predictive analytics, improving patient outcomes and operational efficiency.
- **Finance**: AI algorithms analyze market trends, detect fraud, and automate trading, enhancing decision-making processes.
- **Transportation**: Autonomous vehicles leverage AI for navigation and safety, while AI systems optimize traffic management and logistics.
- **Entertainment**: Streaming services use AI to recommend content based on user preferences, while video games employ AI for more realistic and adaptive gameplay.

#### Challenges and Ethical Considerations
Despite its potential, AI poses several challenges:
- **Bias and Fairness**: AI systems can perpetuate existing biases present in training data, leading to unfair outcomes.
- **Privacy Concerns**: The use of AI in data collection raises significant privacy issues, particularly in surveillance and personal data usage.
- **Job Displacement**: Automation driven by AI may lead to job losses in certain sectors, necessitating workforce retraining and adaptation.

#### Future Outlook
The future of AI is promising, with ongoing research aimed at creating more generalizable and ethical AI systems. Innovations in explainable AI (XAI) seek to make AI decision-making processes more transparent. As AI continues to evolve, collaboration between technologists, policymakers, and ethicists will be crucial to harness its benefits while mitigating risks.

#### Conclusion
AI is transforming industries and society at large, offering significant opportunities for innovation and efficiency. However, it is essential to address the ethical and societal implications of AI to ensure its responsible development and deployment. As we move forward, a balanced approach will be key to maximizing the benefits of AI while minimizing its potential drawbacks.

Here we are making two functions, `extract_fact` to pull out the main content of our text and `replace_word` that will replace AI with Skynet!

In [25]:
def extract_fact(x):
    if "\n\n" in x:
        return "\n".join(x.split("\n\n")[1:])
    else:
        return x

old_word = "AI"
new_word = "skynet"

def replace_word(x):
    return x.replace(old_word, new_word)

Lets wrap these functions and see what the output is!

In [26]:
extract_fact_runnable = RunnableLambda(extract_fact)
replace_word_runnable = RunnableLambda(replace_word)

In [27]:
chain = prompt | llm | output_parser | extract_fact_runnable | replace_word_runnable

In [28]:
result = chain.invoke("retrieval augmented generation")
display(Markdown(result))

#### Introduction
Retrieval-Augmented Generation (RAG) is an innovative approach that combines the strengths of information retrieval and natural language generation. This method enhances the capabilities of language models by allowing them to access external knowledge sources, thereby improving the accuracy and relevance of generated responses.
#### Concept Overview
RAG operates on the principle of integrating a retrieval mechanism with a generative model. The process typically involves two main components:
1. **Retrieval Component**: This part of the system retrieves relevant documents or pieces of information from a large corpus based on a given query. It utilizes techniques such as vector embeddings and similarity search to identify the most pertinent data.
2. **Generation Component**: Once the relevant information is retrieved, the generative model (often based on architectures like Transformers) processes this data to produce coherent and contextually appropriate text. The model can leverage the retrieved information to enhance its responses, making them more informative and accurate.
#### Advantages
- **Enhanced Knowledge Access**: RAG allows models to tap into vast external knowledge bases, overcoming the limitations of fixed training data.
- **Improved Accuracy**: By grounding responses in real-world data, RAG can provide more precise answers, especially for niche or specialized queries.
- **Dynamic Learning**: The retrieval mechanism enables the model to stay updated with the latest information without requiring retraining.
#### Applications
RAG has a wide range of applications, including:
- **Question Answering**: Providing accurate answers to user queries by retrieving relevant documents.
- **Chatbots and Virtual Assistants**: Enhancing conversational agents with up-to-date information.
- **Content Creation**: Assisting in generating articles, reports, or summaries based on retrieved data.
#### Challenges
Despite its advantages, RAG faces several challenges:
- **Retrieval Quality**: The effectiveness of the system heavily relies on the quality of the retrieval component. Poor retrieval can lead to irrelevant or incorrect information being used in generation.
- **Complexity**: Integrating retrieval and generation processes can complicate system design and increase computational requirements.
- **Bias and Misinformation**: If the retrieval corpus contains biased or inaccurate information, it can adversely affect the generated output.
#### Conclusion
Retrieval-Augmented Generation represents a significant advancement in the field of natural language processing. By effectively combining retrieval and generation, RAG systems can produce more accurate, relevant, and contextually aware responses. As research and development in this area continue, RAG is poised to play a crucial role in the evolution of intelligent conversational agents and information systems. Future work will likely focus on improving retrieval mechanisms, addressing biases, and enhancing the overall efficiency of RAG systems.

Those are our `RunnableLambda` functions. It's worth noting that all inputs to these functions are expected to be a SINGLE arguments. If you have a function that accepts multiple arguments, you can input a dictionary with keys, then unpack them inside the function.

## LCEL `RunnableParallel` and `RunnablePassthrough`

LCEL provides us with various `Runnable` classes that allow us to control the flow of data and execution order through our chains. Two of these are `RunnableParallel` and `RunnablePassthrough`.

* `RunnableParallel` — allows us to run multiple `Runnable` instances in parallel. Acting almost as a Y-fork in the chain.

* `RunnablePassthrough` — allows us to pass through a variable to the next `Runnable` without modification.

To see these runnables in action, we will create two data sources, each source provides specific information but to answer the question we will need both to fed to the LLM.

In [29]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

embedding = OpenAIEmbeddings()

vecstore_a = DocArrayInMemorySearch.from_texts(
    [
        "half the info is here",
        "DeepSeek-V3 was released in December 2024"
    ],
    embedding=embedding
)
vecstore_b = DocArrayInMemorySearch.from_texts(
    [
        "the other half of the info is here",
        "the DeepSeek-V3 LLM is a mixture of experts model with 671B parameters"
    ],
    embedding=embedding
)

<ipython-input-29-7a4850ad4e22>:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()
/usr/local/lib/python3.11/dist-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


Here you can see the prompt does have three inputs, two for context and one for the question itself.

In [30]:
prompt_str = """Using the context provided, answer the user's question.
Context:
{context_a}
{context_b}
"""

In [31]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(prompt_str),
    HumanMessagePromptTemplate.from_template("{question}")
])

Here we are wrapping our vector stores as retrievers so they can be fitted into one big retrieval variable to be used by the prompt.

In [32]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

retriever_a = vecstore_a.as_retriever()
retriever_b = vecstore_b.as_retriever()

retrieval = RunnableParallel(
    {
        "context_a": retriever_a, "context_b": retriever_b, "question": RunnablePassthrough()
    }
)

The chain we'll be constructing will look something like this:

![](https://github.com/aurelio-labs/langchain-course/blob/main/assets/lcel-flow.png?raw=1)

In [33]:
chain = retrieval | prompt | llm | output_parser

We `invoke` it as usual.

In [34]:
result = chain.invoke(
    "what architecture does the model DeepSeek released in december use?"
)
result

'The DeepSeek-V3 model, released in December 2024, is a mixture of experts model with 671 billion parameters.'

With that we've seen how we can use `RunnableParallel` and `RunnablePassthrough` to control the flow of data and execution order through our chains.

---